## Python for Finance 6 - Best Stocks to Buy Now

In [ ]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

#Statsmodels is a great library we can use to run regressions.
import statsmodels.api as sm
# Seaborn extends the capabilities of Matplotlib
import seaborn as sns
# Used for calculating regressions
from statsmodels.tsa.ar_model import AutoReg, ar_select_order

#### Dates & Other Constants

In [ ]:
# Define path to files
PATH = "../data/stock-list/"

# Start date defaults
ST_YEAR = 2017
ST_MONTH = 1
ST_DAY = 1
ST_DATE_STR = f"{ST_YEAR}-{ST_MONTH}-{ST_DAY}"
ST_DATE_DATETIME = dt.datetime(ST_YEAR, ST_MONTH, ST_DAY)

# End date defaults
EN_YEAR = 2021
EN_MONTH = 12
EN_DAY = 31
EN_DATE_STR = f"{EN_YEAR}-{EN_MONTH}-{EN_DAY}"
EN_DATE_DATETIME = dt.datetime(EN_YEAR, EN_MONTH, EN_DAY)

#### Hold Stocks not Downloaded

In [ ]:
stocks_not_downloaded = []
missing_stocks = []

#### Get Stock File Names in a List

In [ ]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers

tickers.sort()
len(tickers)

#### Save Stock Data to CSV

In [ ]:
# Function that gets a dataframe by providing a ticker and starting date
def save_to_csv_from_yahoo(ticker, syear, smonth, sday, eyear, emonth, eday):
    # Defines the time periods to use
    start = dt.datetime(syear, smonth, sday)
    end = dt.datetime(eyear, emonth, eday)
    
    try:
        print("Get Data for : ", ticker)
        # Reads data into a dataframe
        df = web.DataReader(ticker, 'yahoo', start, end)['Adj Close']
    
        # Wait 10 seconds
        time.sleep(5)
    
        # Save data to a CSV file Windows
        df.to_csv('../data/update/' + ticker + '.csv')
        
    except Exception as ex:
        stocks_not_downloaded.append(ticker)
        print("Couldn't Get Data for :", ticker)

In [ ]:
for x in range(0, 500):
    save_to_csv_from_yahoo(tickers[x], 2022, 1, 1, 2022, 1, 20)
print("Finished")
stocks_not_downloaded

In [ ]:
for x in range(500, 1000):
    save_to_csv_from_yahoo(tickers[x], 2022, 1, 1, 2022, 1, 20)
print("Finished")
stocks_not_downloaded

In [ ]:
for x in range(1000, 1500):
    save_to_csv_from_yahoo(tickers[x], 2022, 1, 1, 2022, 1, 20)
print("Finished")
stocks_not_downloaded

In [ ]:
for x in range(1500, 2000):
    save_to_csv_from_yahoo(tickers[x], 2022, 1, 1, 2022, 1, 20)
print("Finished")
stocks_not_downloaded

In [ ]:
for x in range(2000, 2888):
    save_to_csv_from_yahoo(tickers[x], 2022, 1, 1, 2022, 1, 20)
print("Finished")
stocks_not_downloaded

In [ ]:
for x in missing_stocks:
    save_to_csv_from_yahoo(x, 2022, 1, 1, 2022, 1, 20)
print("Finished")
stocks_not_downloaded

#### Function that Returns a Dataframe from a CSV

In [ ]:
def get_df_from_csv(folder, ticker):
    try:
        df = pd.read_csv(folder + ticker + '.csv', index_col='Date', 
                         parse_dates=True)
    except FileNotFoundError:
        pass
        print("File Doesn't Exist")
    else:
        return df

#### Merge New Data

In [ ]:
def update_stock_data(ticker):
    # For Windows 
    up_folder = '../data/update/'

    # For Windows 
    stock_folder = PATH

    update_df = get_df_from_csv(up_folder, ticker)
    update_df

    # Get original dataframe
    orig_df = get_df_from_csv(stock_folder, ticker)

    # Drop the unnamed column
    orig_df.drop(orig_df.columns[orig_df.columns.str.contains('unnamed',case = False)],
          axis = 1, inplace = True)

    # Drop the daily_return column
    orig_df = orig_df.drop('daily_return', 1)

    join_df = pd.concat([orig_df, update_df])
    return join_df

#### Save Dataframe to CSV

In [ ]:
def save_dataframe_to_csv(df, folder, ticker):
    df.to_csv(folder + ticker + '.csv')

#### Update Files with New Data

In [ ]:
def update_stocks(stock_folder):
    
    for x in tickers:
        try:
            print("Working on :", x)
            new_df = update_stock_data(x)
            save_dataframe_to_csv(new_df, stock_folder, x)
        except Exception as ex:
            print(ex)

In [ ]:
# For Windows
update_stocks(PATH)

#### Add Daily Return to Dataframe

In [ ]:
# We calculate a percentage rate of return for each day to compare investments.
# Simple Rate of Return = (End Price - Beginning Price) / Beginning Price OR (EP / BP) - 1

# Shift provides the value from the previous day
# NaN is displayed because there was no previous day price for the 1st calculation
def add_daily_return_to_df(df, ticker):
    df['daily_return'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    # Save data to a CSV file
    # For Windows
    df.to_csv(PATH + ticker + '.csv')
    
    return df  

#### Update Daily Returns

In [ ]:
for x in tickers:
    try:
        print("Working on :", x)
        new_df = get_df_from_csv(PATH, x)
        new_df = add_daily_return_to_df(new_df, x)
        save_dataframe_to_csv(new_df, PATH, x)
    except Exception as ex:
        print(ex)

#### Regression Time Series Example with AutoReg

In [ ]:
def calc_projected_roi(ticker):
    a_df = get_df_from_csv(PATH, ticker)

    a_df = a_df.asfreq('d') # Change frequency to day
    a_df.index # Check frequency
    a_df = a_df.fillna(method='ffill') # Fill missing values

    # Delete unnamed column
    a_df.drop(a_df.columns[a_df.columns.str.contains('unnamed',case = False)],
          axis = 1, inplace = True)

    # Delete daily return column
    a_df = a_df.drop(['daily_return'], axis=1)
    
    # Figure out optimum lags which will be 1 or 2 for this data set
    lags = ar_select_order(a_df, maxlag=30)

    # Create our model using whole data set
    model = AutoReg(a_df['Adj Close'], lags.ar_lags)
    model_fit = model.fit()

    # Define training and testing area
    print("Length :",len(a_df)) # 1712 observations
    train_df = a_df.iloc[50:1369] # 80% minus 1st 50
    test_df = a_df.iloc[1369:] # Last 20%

    # Define training model for 500 days (Play with Number & Test)
    # and White's covariance estimator
    train_model = AutoReg(a_df['Adj Close'], 500).fit(cov_type="HC0")

    # Define start and end for prediction 
    start = len(train_df)
    end = len(train_df) + len(test_df) - 1

    prediction = train_model.predict(start=start, end=end, dynamic=True)

    # Predict 160 days into the future
    forecast = train_model.predict(start=end, end=end+60, dynamic=True)

    # Get starting price of prediction
    s_price = forecast.head(1).iloc[0]

    # Get the last price of prediction
    e_price = forecast.iloc[-1]

    # Get return over prediction
    return (e_price - s_price) / s_price

In [ ]:
def get_proj_rois():
    # Will hold all tickers & stock rois
    ticker = []
    roi = []
    
    for x in tickers:
        print("Working on :", x)
        try:
            the_roi = calc_projected_roi(x)
        except Exception as ex:
            print("Stock Data Corrupted")
        else:
            ticker.append(x)
            print("ROI :", the_roi)
            roi.append(the_roi)
        
    return pd.DataFrame({'Ticker':ticker, 'ROI':roi})

In [ ]:
proj_roi_df = get_proj_rois()
proj_roi_df

In [ ]:
proj_roi_df.sort_values(by=['ROI'], ascending=False)[0:20]